# **Optimizing Bluebikes Operations through Spatial and Demand Forecasting**

Team B05: Mohamad Gong, Marcus Shi, Tianqi Sun, Tzu-Jen Chen, Arshdeep Oberoi, Xiaoqing Ye



## Executive Summary

### Objective

* Analyze Bluebikes trip data (May 2018 – September 2025) to understand spatial and temporal patterns of ridership across the Boston region.

* Examine how user type, trip duration, and bike type (classic vs e-bike) influence ride behavior and system utilization.

* Assess weather impacts on demand using integrated daily NOAA data.

* Identify high-throughput stations and key origin–destination pairs to support rebalancing and capacity planning.

* Prepare neighborhood-level and landmark-based extensions for spatial forecasting and system optimization.

### Findings

* Ridership is heavily concentrated along the MIT–Harvard corridor and major transit nodes such as Central Square and Beacon St, confirming the role of dense, mixed-use areas in sustaining demand.

* Short trips dominate the system: nearly 75 % of rides last under 20 minutes, reflecting commuter and errand-based use rather than leisure.

* Members generate roughly three times as many rides as casual users, but casuals complete round trips four times more often, underscoring distinct motivations between groups.

* Top station pairs are clustered around MIT and nearby Cambridge connectors, indicating strong local circulation and predictable rebalancing corridors.

* Weather analysis shows substantial drops in ridership during hazardous or extreme-temperature days, suggesting clear value in weather-responsive service adjustments.

### Methods

*   We cleaned and combined trip, station, and weather data to create a reliable, analysis-ready view of the Bluebikes system.

*   We used structured database queries to summarize ridership, trip duration, user types, and seasonal patterns across the network.

*   We linked trips to detailed station attributes and local weather conditions to understand where, when, and under what circumstances usage is highest or most constrained.

*   We calculated key percentages and comparisons (e.g., share of trips by user type, bike type, season, and corridor) to highlight where operational changes would have the greatest impact.

### Recommendations

* Increase dock capacity and rebalancing frequency at top stations along the Mass Ave spine to prevent shortages and excess returns.

* Expand e-bike deployment in high-demand areas and near universities to capitalize on short-trip peaks.

* Develop dashboards and maps that monitor trip volume, dock utilization, and weather-adjusted demand in real time.

* Integrate neighborhood and landmark layers to uncover under-served zones and guide future station placement.

* Address missing and null data fields before predictive modeling and consider Python-based tools for spatial joins and geo-processing to reduce SQL complexity.

## Problem Definition

Our project aims to support Boston’s Bluebikes system in improving station placement, bike availability, and overall operational efficiency through data-driven spatial and temporal forecasting. We will:
*   Identify ridership trends across stations and time periods.
*   Quantify how weather conditions, MBTA rapid-transit accessibility, and surrounding land use influence demand.
*   Detect underutilized stations and areas with unmet demand.
*   Develop predictive models and spatial visualizations to guide rebalancing and future expansion strategies.


## Motivation

Using interoperable public datasets and proven analytics, we will deliver actionable insights to optimize Bluebikes’ station placement, rebalancing, and service reliability for Boston’s mobility stakeholders. As BU students who sometimes ride Bluebikes, we’re personally invested in improvements that make the system more convenient, equitable, and dependable.

# **1. Data Sources**

**Bluebikes Trip Data:**

Trip-level records of bike-share rides from Boston’s public bike-share system, originally launched as Hubway in 2011 and rebranded as Bluebikes in May 2018 following a new sponsorship agreement with Blue Cross Blue Shield of Massachusetts. The dataset documents individual trips between docking stations, including trip start and end times, station names and IDs, bike IDs, and trip durations, published as monthly zipped CSV files. For this project, we include data only from May 2018 onward to align with the Bluebikes era, ensuring a consistent system brand, operator, station network, and pricing/technology environment; earlier Hubway-period data are excluded for analytical continuity.

The data has been processed to remove trips that are taken by staff as they service and inspect the system; and any trips that were below 60 seconds in length (potentially false starts or users trying to re-dock a bike to ensure it was secure).

**Coverage:** May 2018–Sep 2025

**Format:** CSV (monthly zip archives)

**Size:** ~2.4 GB

**Link:** https://s3.amazonaws.com/hubway-data/index.html


---


**Bluebikes Station List:**

Official list of active Bluebikes docking stations, published and maintained by the Bluebikes program. Each record includes station ID, name, geographic coordinates, municipality, and total number of docks. Designed to support spatial joins with trip and demographic data.

**Coverage:** August 2025 (current active station network)

**Format:** CSV

**Size:** 0.08 MB

**Link:** https://bluebikes.com/system-data



---



**Weather Data:**

Daily and hourly weather observations from official NWS/NCEI stations, including temperature, wind, precipitation, snowfall, and coded weather conditions. Used to construct one record per day with key metrics (e.g., average temperature, rain/snow indicators, and simple hazard flags) for integrating weather context into ridership analyses.

**Coverage:** Station-specific, multi-year through 2025

**Format:** CSV (hourly + daily summaries)

**Size:** Varies by station and period (~5–50 MB per station)

**Link:** https://www.ncei.noaa.gov/cdo-web/datatools/

# **2. Data Importing and Cleaning**

## 2.1 Importing Data

For this project, all analysis was done on Google Cloud using Cloud Storage for raw data and BigQuery for warehousing and queries. Most tables were imported via the BigQuery web interface for simplicity. The exception was the trips data: it spans 89 monthly CSV files with occasional column-name/schema differences, so we used Cloud Shell to bulk load from Cloud Storage into a single BigQuery table. This approach (using `gsutil` + `bq` load with schema autodetection and allowed schema updates) handled the file count and minor schema drift reliably and reproducibly.

```
gcloud config set project ba775-fall25-b05

PROJECT_ID=ba775-fall25-b05
DATASET=bluebikes
TABLE=trips
BUCKET=gs://ba775-fall25-b05/trip_history

# create/overwrite the table using the first file (sorted so 201805 loads first)
FIRST_URI=$(gsutil ls "$BUCKET/*.csv" | sort | head -n 1)
bq load \
  --source_format=CSV \
  --autodetect \
  --skip_leading_rows=1 \
  --replace \
  "$PROJECT_ID:$DATASET.$TABLE" \
  "$FIRST_URI"

# append the rest, allowing small schema changes over time
for URI in $(gsutil ls "$BUCKET/*.csv" | sort | tail -n +2); do
  bq load \
    --source_format=CSV \
    --autodetect \
    --skip_leading_rows=1 \
    --schema_update_option=ALLOW_FIELD_ADDITION \
    --schema_update_option=ALLOW_FIELD_RELAXATION \
    "$PROJECT_ID:$DATASET.$TABLE" \
    "$URI"
done
```



## 2.2 Data Cleaning

### Trips Data

The first step would be to inspect the trips table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.bluebikes.trips`
LIMIT
  5
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=5, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.bluebikes`.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name = 'trips'
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=29, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

After inspecting the combined trips table, we found duplicate columns with slightly different names (e.g., `start_station_name` vs. `start station name`). This inconsistency comes from the monthly source files, which were published as separate tables with varying column names and orders. The next step is to standardize the schema and merge these semantically equivalent columns into single, consistent fields.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE
  `ba775-fall25-b05.ba775_fall25_b05.trips_clean` AS
SELECT
  ride_id,
  rideable_type,
  COALESCE(started_at, starttime) AS started_at,
  COALESCE(ended_at, stoptime) AS ended_at,
  COALESCE( tripduration, SAFE_CAST(TIMESTAMP_DIFF(COALESCE(ended_at, stoptime), COALESCE(started_at, starttime), SECOND) AS INT64) ) AS trip_duration_sec,
  COALESCE(start_station_id, CAST(`start station id` AS STRING)) AS start_station_id,
  COALESCE(start_station_name, `start station name`) AS start_station_name,
  COALESCE(start_lat, `start station latitude`) AS start_lat,
  COALESCE(start_lng, `start station longitude`) AS start_lng,
  COALESCE(end_station_id, CAST(`end station id` AS STRING)) AS end_station_id,
  COALESCE(end_station_name, `end station name`) AS end_station_name,
  COALESCE(end_lat, `end station latitude`) AS end_lat,
  COALESCE(end_lng, `end station longitude`) AS end_lng,
  member_casual,
  usertype,
  CAST(bikeid AS STRING) AS bike_id,
  `postal code` AS postal_code,
  `birth year` AS birth_year,
  gender
FROM
  `ba775-fall25-b05.bluebikes.trips`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
LIMIT
  5
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=5, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
where end_station_id is null
LIMIT
  5
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=5, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  EXTRACT(YEAR  FROM DATE(started_at)) AS yr,
  EXTRACT(MONTH FROM DATE(started_at)) AS mo,

  SUM(CASE WHEN ride_id IS NULL THEN 1 ELSE 0 END) AS ride_id_nulls,
  SUM(CASE WHEN ride_id IS NOT NULL THEN 1 ELSE 0 END) AS ride_id_not_nulls,

  SUM(CASE WHEN rideable_type IS NULL THEN 1 ELSE 0 END) AS rideable_type_nulls,
  SUM(CASE WHEN rideable_type IS NOT NULL THEN 1 ELSE 0 END) AS rideable_type_not_nulls,

  SUM(CASE WHEN started_at IS NULL THEN 1 ELSE 0 END) AS started_at_nulls,
  SUM(CASE WHEN started_at IS NOT NULL THEN 1 ELSE 0 END) AS started_at_not_nulls,

  SUM(CASE WHEN ended_at IS NULL THEN 1 ELSE 0 END) AS ended_at_nulls,
  SUM(CASE WHEN ended_at IS NOT NULL THEN 1 ELSE 0 END) AS ended_at_not_nulls,

  SUM(CASE WHEN trip_duration_sec IS NULL THEN 1 ELSE 0 END) AS trip_duration_sec_nulls,
  SUM(CASE WHEN trip_duration_sec IS NOT NULL THEN 1 ELSE 0 END) AS trip_duration_sec_not_nulls,

  SUM(CASE WHEN start_station_id IS NULL THEN 1 ELSE 0 END) AS start_station_id_nulls,
  SUM(CASE WHEN start_station_id IS NOT NULL THEN 1 ELSE 0 END) AS start_station_id_not_nulls,

  SUM(CASE WHEN start_station_name IS NULL THEN 1 ELSE 0 END) AS start_station_name_nulls,
  SUM(CASE WHEN start_station_name IS NOT NULL THEN 1 ELSE 0 END) AS start_station_name_not_nulls,

  SUM(CASE WHEN start_lat IS NULL THEN 1 ELSE 0 END) AS start_lat_nulls,
  SUM(CASE WHEN start_lat IS NOT NULL THEN 1 ELSE 0 END) AS start_lat_not_nulls,

  SUM(CASE WHEN start_lng IS NULL THEN 1 ELSE 0 END) AS start_lng_nulls,
  SUM(CASE WHEN start_lng IS NOT NULL THEN 1 ELSE 0 END) AS start_lng_not_nulls,

  SUM(CASE WHEN end_station_id IS NULL THEN 1 ELSE 0 END) AS end_station_id_nulls,
  SUM(CASE WHEN end_station_id IS NOT NULL THEN 1 ELSE 0 END) AS end_station_id_not_nulls,

  SUM(CASE WHEN end_station_name IS NULL THEN 1 ELSE 0 END) AS end_station_name_nulls,
  SUM(CASE WHEN end_station_name IS NOT NULL THEN 1 ELSE 0 END) AS end_station_name_not_nulls,

  SUM(CASE WHEN end_lat IS NULL THEN 1 ELSE 0 END) AS end_lat_nulls,
  SUM(CASE WHEN end_lat IS NOT NULL THEN 1 ELSE 0 END) AS end_lat_not_nulls,

  SUM(CASE WHEN end_lng IS NULL THEN 1 ELSE 0 END) AS end_lng_nulls,
  SUM(CASE WHEN end_lng IS NOT NULL THEN 1 ELSE 0 END) AS end_lng_not_nulls,

  SUM(CASE WHEN member_casual IS NULL THEN 1 ELSE 0 END) AS member_casual_nulls,
  SUM(CASE WHEN member_casual IS NOT NULL THEN 1 ELSE 0 END) AS member_casual_not_nulls,

  SUM(CASE WHEN usertype IS NULL THEN 1 ELSE 0 END) AS usertype_nulls,
  SUM(CASE WHEN usertype IS NOT NULL THEN 1 ELSE 0 END) AS usertype_not_nulls,

  SUM(CASE WHEN bike_id IS NULL THEN 1 ELSE 0 END) AS bike_id_nulls,
  SUM(CASE WHEN bike_id IS NOT NULL THEN 1 ELSE 0 END) AS bike_id_not_nulls,

  SUM(CASE WHEN postal_code IS NULL THEN 1 ELSE 0 END) AS postal_code_nulls,
  SUM(CASE WHEN postal_code IS NOT NULL THEN 1 ELSE 0 END) AS postal_code_not_nulls,

  SUM(CASE WHEN birth_year IS NULL THEN 1 ELSE 0 END) AS birth_year_nulls,
  SUM(CASE WHEN birth_year IS NOT NULL THEN 1 ELSE 0 END) AS birth_year_not_nulls,

  SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS gender_nulls,
  SUM(CASE WHEN gender IS NOT NULL THEN 1 ELSE 0 END) AS gender_not_nulls

FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY yr, mo
ORDER BY yr, mo;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=89, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

After inspecting the data, the following steps will be taken next:

`ride_id`: This field was not recorded until April 2023. For records where it is missing, a value will be generated using a deterministic hash of timestamps, station identifiers, and bike_id so that identifiers remain stable across rebuilds.

`rideable_type` and `bike_id`: These two fields were not recorded until April 2023. They will be kept exactly as present in the source data, and no values will be inferred for earlier periods.

Datetime fields (`started_at`, `ended_at`): These fields are complete and will be left unchanged.

Start-station fields (`start_station_id`, `start_station_name`, `start_lat`, `start_lng`): Because trips without a beginning cannot be used in analysis, rows with any missing start-station value will be removed.

End-station fields (`end_station_id`, `end_station_name`, `end_lat`, `end_lng`): Because trips without a destination cannot be used in destination analysis, rows with any missing end-station value will be removed.

User type: A single unified column named `user_type` will be created. Values will be taken from `member_casual` when available; otherwise, legacy `usertype` will be mapped so that Subscriber and Member become member, and Customer and Casual become casual. The original two columns may then be dropped from the final table.

`postal_code`: Because this field is largely missing from May 2020 through April 2023 and is redundant with latitude and longitude, it will be dropped.

`birth_year` and `gender`: Because these fields stopped being recorded in May 2020, they will be dropped.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE
  `ba775-fall25-b05.ba775_fall25_b05.trips_clean` AS
WITH
  base AS (
  SELECT
    COALESCE( ride_id, TO_HEX(SHA256(CONCAT( CAST(started_at AS STRING), '|', CAST(ended_at AS STRING), '|', IFNULL(start_station_id, ''), '|', IFNULL(end_station_id, ''), '|', IFNULL(bike_id, '') ))) ) AS ride_id,
    rideable_type,
    bike_id,
    started_at,
    ended_at,
    trip_duration_sec,
    start_station_id,
    start_station_name,
    start_lat,
    start_lng,
    end_station_id,
    end_station_name,
    end_lat,
    end_lng,
    COALESCE( member_casual,
      CASE
        WHEN LOWER(usertype) IN ('subscriber', 'member') THEN 'member'
        WHEN LOWER(usertype) IN ('customer',
        'casual') THEN 'casual'
        ELSE NULL
    END
      ) AS user_type
  FROM
    `ba775-fall25-b05.ba775_fall25_b05.trips_clean` )
SELECT
  *
FROM
  base
WHERE
  start_station_id IS NOT NULL
  AND start_station_name IS NOT NULL
  AND start_lat IS NOT NULL
  AND start_lng IS NOT NULL
  AND end_station_id IS NOT NULL
  AND end_station_name IS NOT NULL
  AND end_lat IS NOT NULL
  AND end_lng IS NOT NULL;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

### Stations Data

When inspecting the stations table it appears there is a legacy numbering, and that is backed up when inspecting the station IDs in the trips table.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.bluebikes.stations`
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=572, table_html='<table border="1" class="dataframe table table-striped ta…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.bluebikes`.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name = 'stations'
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=8, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

For that we will replace the old station ID with the new one if possible, and add Seasonal Status and Total Docks.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE `ba775-fall25-b05.ba775_fall25_b05.trips_clean` AS
SELECT
  t.ride_id,
  t.rideable_type,
  t.bike_id,
  t.started_at,
  t.ended_at,
  t.trip_duration_sec,

  COALESCE(s_start.Number, t.start_station_id) AS start_station_id,
  t.start_station_name,
  t.start_lat,
  t.start_lng,

  COALESCE(s_end.Number, t.end_station_id)     AS end_station_id,
  t.end_station_name,
  t.end_lat,
  t.end_lng,

  t.user_type,

  s_start.`Seasonal Status` AS start_seasonal_status,
  CAST(s_start.`Total Docks` AS INT64) AS start_total_docks,
  s_end.`Seasonal Status`   AS end_seasonal_status,
  CAST(s_end.`Total Docks` AS INT64)   AS end_total_docks
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean` t
LEFT JOIN `ba775-fall25-b05.bluebikes.stations` s_start
  ON t.start_station_name = s_start.NAME
 AND ROUND(t.start_lat, 4) = ROUND(s_start.Lat, 4)
 AND ROUND(t.start_lng, 4) = ROUND(s_start.Long, 4)
LEFT JOIN `ba775-fall25-b05.bluebikes.stations` s_end
  ON t.end_station_name = s_end.NAME
 AND ROUND(t.end_lat, 4) = ROUND(s_end.Lat, 4)
 AND ROUND(t.end_lng, 4) = ROUND(s_end.Long, 4);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH unmapped AS (
  SELECT start_station_id AS station_id
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean` t
  LEFT JOIN `ba775-fall25-b05.bluebikes.stations` s
    ON t.start_station_id = s.Number
  WHERE s.Number IS NULL
  UNION ALL
  SELECT end_station_id
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean` t
  LEFT JOIN `ba775-fall25-b05.bluebikes.stations` s
    ON t.end_station_id = s.Number
  WHERE s.Number IS NULL
)
SELECT station_id, COUNT(*) AS trips
FROM unmapped
GROUP BY station_id
ORDER BY trips DESC;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=200, table_html='<table border="1" class="dataframe table table-striped ta…

Computation deferred. Computation will process 0 Bytes

As it appears, many stations did not match the stations table, and that is acceptable. The trips still retain station name and coordinates (latitude and longitude), which allow station-level analysis. The only information missing for unmatched rows is Seasonal Status and Total Docks, which are supplementary fields; the core trip records remain usable for the analysis.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE
  `ba775-fall25-b05.ba775_fall25_b05.trips_clean` AS
WITH
  src AS (
  SELECT
    *
  FROM
    `ba775-fall25-b05.ba775_fall25_b05.trips_clean` )
SELECT
  ride_id,
  started_at,
  ended_at,
  trip_duration_sec,
  rideable_type,
  bike_id,
  user_type,
  start_station_id,
  start_station_name,
  start_lat,
  start_lng,
  start_seasonal_status,
  start_total_docks,
  end_station_id,
  end_station_name,
  end_lat,
  end_lng,
  end_seasonal_status,
  end_total_docks
FROM
  src;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

Inspecting the final table after cleaning.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  EXTRACT(YEAR FROM started_at) AS year,
  EXTRACT(MONTH FROM started_at) AS month,

  -- ride_id
  COUNTIF(ride_id IS NULL) AS ride_id_null,
  COUNTIF(ride_id IS NOT NULL) AS ride_id_not_null,

  -- started_at
  COUNTIF(started_at IS NULL) AS started_at_null,
  COUNTIF(started_at IS NOT NULL) AS started_at_not_null,

  -- ended_at
  COUNTIF(ended_at IS NULL) AS ended_at_null,
  COUNTIF(ended_at IS NOT NULL) AS ended_at_not_null,

  -- trip_duration_sec
  COUNTIF(trip_duration_sec IS NULL) AS trip_duration_sec_null,
  COUNTIF(trip_duration_sec IS NOT NULL) AS trip_duration_sec_not_null,

  -- rideable_type
  COUNTIF(rideable_type IS NULL) AS rideable_type_null,
  COUNTIF(rideable_type IS NOT NULL) AS rideable_type_not_null,

  -- bike_id
  COUNTIF(bike_id IS NULL) AS bike_id_null,
  COUNTIF(bike_id IS NOT NULL) AS bike_id_not_null,

  -- user_type
  COUNTIF(user_type IS NULL) AS user_type_null,
  COUNTIF(user_type IS NOT NULL) AS user_type_not_null,

  -- start_station_id
  COUNTIF(start_station_id IS NULL) AS start_station_id_null,
  COUNTIF(start_station_id IS NOT NULL) AS start_station_id_not_null,

  -- start_station_name
  COUNTIF(start_station_name IS NULL) AS start_station_name_null,
  COUNTIF(start_station_name IS NOT NULL) AS start_station_name_not_null,

  -- start_lat / start_lng
  COUNTIF(start_lat IS NULL) AS start_lat_null,
  COUNTIF(start_lat IS NOT NULL) AS start_lat_not_null,

  COUNTIF(start_lng IS NULL) AS start_lng_null,
  COUNTIF(start_lng IS NOT NULL) AS start_lng_not_null,

  -- start_seasonal_status
  COUNTIF(start_seasonal_status IS NULL) AS start_seasonal_status_null,
  COUNTIF(start_seasonal_status IS NOT NULL) AS start_seasonal_status_not_null,

  -- start_total_docks
  COUNTIF(start_total_docks IS NULL) AS start_total_docks_null,
  COUNTIF(start_total_docks IS NOT NULL) AS start_total_docks_not_null,

  -- end_station_id
  COUNTIF(end_station_id IS NULL) AS end_station_id_null,
  COUNTIF(end_station_id IS NOT NULL) AS end_station_id_not_null,

  -- end_station_name
  COUNTIF(end_station_name IS NULL) AS end_station_name_null,
  COUNTIF(end_station_name IS NOT NULL) AS end_station_name_not_null,

  -- end_lat / end_lng
  COUNTIF(end_lat IS NULL) AS end_lat_null,
  COUNTIF(end_lat IS NOT NULL) AS end_lat_not_null,

  COUNTIF(end_lng IS NULL) AS end_lng_null,
  COUNTIF(end_lng IS NOT NULL) AS end_lng_not_null,

  -- end_seasonal_status
  COUNTIF(end_seasonal_status IS NULL) AS end_seasonal_status_null,
  COUNTIF(end_seasonal_status IS NOT NULL) AS end_seasonal_status_not_null,

  -- end_total_docks
  COUNTIF(end_total_docks IS NULL) AS end_total_docks_null,
  COUNTIF(end_total_docks IS NOT NULL) AS end_total_docks_not_null

FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY year, month
ORDER BY year, month;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=89, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

* Core trip fields (`ride_id`, timestamps, duration, user & station IDs/names, lat/lng) are fully populated with no NULL values across all months.
* Schema change in 2023-04: older data has `bike_id` filled and `rideable_type` always NULL; newer data has `rideable_type` filled and `bike_id` always NULL.
* Station metadata fields (`start_seasonal_status`, `end_seasonal_status`, `start_total_docks`, `end_total_docks`) have partial coverage and often go NULL together.
* The share of NULLs in these station metadata fields increases in more recent months, so analyses using capacity/seasonality should treat these as “unknown” or filter them out.

### Climate Data

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  *
FROM
  `ba775-fall25-b05.weather`.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name = 'weather_raw'
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=11, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

From this, we generated one record per day with clear indicators for conditions like rain, snow, wind, or sun, and optional flags for extreme or hazardous weather. This cleaned dataset allows us to easily connect weather patterns with other data sources, such as transportation activity.

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE `ba775-fall25-b05.ba775_fall25_b05.weather_daily_summary` AS
WITH src AS (
  SELECT
    DATE(DATE) AS date,
    SAFE_CAST(DailyAverageDryBulbTemperature  AS FLOAT64) AS avg_temp,
    SAFE_CAST(DailyMaximumDryBulbTemperature AS FLOAT64) AS max_temp,
    SAFE_CAST(DailyMinimumDryBulbTemperature AS FLOAT64) AS min_temp,
    SAFE_CAST(DailyPrecipitation             AS FLOAT64) AS precip,
    SAFE_CAST(DailySnowfall                  AS FLOAT64) AS snowfall,
    SAFE_CAST(DailySnowDepth                 AS FLOAT64) AS snow_depth,
    SAFE_CAST(DailyAverageWindSpeed          AS FLOAT64) AS avg_wind,
    SAFE_CAST(DailyPeakWindSpeed             AS FLOAT64) AS peak_wind,
    SAFE_CAST(DailySustainedWindSpeed        AS FLOAT64) AS sust_wind,
    UPPER(IFNULL(TRIM(DailyWeather), '')) AS wx
  FROM `ba775-fall25-b05.weather.weather_raw`
  WHERE DATE IS NOT NULL
),
agg AS (
  SELECT
    date,
    AVG(avg_temp)    AS avg_temp,
    AVG(max_temp)    AS max_temp,
    AVG(min_temp)    AS min_temp,
    AVG(precip)      AS precip,
    AVG(snowfall)    AS snowfall,
    AVG(snow_depth)  AS snow_depth,
    AVG(avg_wind)    AS avg_wind,
    AVG(peak_wind)   AS peak_wind,
    AVG(sust_wind)   AS sust_wind,
    STRING_AGG(DISTINCT wx, ';') AS wx
  FROM src
  GROUP BY date
),
labeled AS (
  SELECT
    date,
    avg_temp,
    CASE
      WHEN snowfall > 0 OR snow_depth > 0 OR REGEXP_CONTAINS(wx, r'SN') THEN 'snow'
      WHEN precip > 0 OR REGEXP_CONTAINS(wx, r'RA|DZ|SH') THEN 'rain'
      WHEN peak_wind >= 25 OR sust_wind >= 20 THEN 'windy'
      WHEN REGEXP_CONTAINS(wx, r'FG|BR|HZ') THEN 'foggy'
      WHEN IFNULL(precip, 0) = 0 AND (wx = '' OR wx IS NULL) THEN 'sunny'
      ELSE 'mixed'
    END AS status,
    CASE
      WHEN peak_wind >= 40 OR sust_wind >= 25 THEN 'danger_wind'
      WHEN precip >= 1.0 THEN 'danger_heavy_rain'
      WHEN snowfall >= 4 OR snow_depth >= 6 THEN 'danger_snow'
      WHEN avg_temp >= 95 THEN 'danger_heat'
      WHEN avg_temp <= 15 THEN 'danger_cold'
      WHEN REGEXP_CONTAINS(wx, r'TS|FC|FZRA|FZDZ|BLSN') THEN 'danger_storm'
      ELSE 'none'
    END AS danger
  FROM agg
)
SELECT
  date,
  avg_temp AS temp,
  status,
  danger
FROM labeled
WHERE avg_temp IS NOT NULL
ORDER BY date;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

# **3. Exploratory Data Analysis**

With the data now standardized and free of major inconsistencies, we can begin exploring it to uncover patterns and relationships that will inform our subsequent modelling. The exploratory analysis is organised into three themes: ridership trends over time, ride characteristics and user behaviour, and weather and seasonal effects. Within each theme we pose concrete questions and use SQL queries on the cleaned tables to answer them. The examples below illustrate the process; your notebook should execute the SQL and display the results so that you can validate assumptions and spot anomalies.

## 3.1 Ridership trends over time

### Q1. How has overall ridership changed by year?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  EXTRACT(YEAR FROM started_at) AS ride_year,
  COUNT(*) AS total_rides
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY ride_year
ORDER BY ride_year;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=8, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

Overall ridership shows steady growth year-over-year until the pandemic period (2020) where usage dipped, followed by a rebound in later years. This indicates Bluebikes’ long-term adoption with short-term disruption from external events.

### Q2. What months are busiest across all years?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  EXTRACT(MONTH FROM started_at) AS month_num,
  FORMAT_DATE('%B', DATE(started_at)) AS month_name,
  COUNT(*) AS total_rides
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY month_num, month_name
ORDER BY total_rides DESC;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=12, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

June-September are consistently the busiest months, aligning with warm weather and tourism peaks. Winter months (December–February) have the lowest activity with cold weather.

## 3.2 Ride characteristics and user behaviour

### Q3. What is the typical trip duration?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  ROUND(AVG(trip_duration_sec) / 60, 2) AS avg_duration_min,
  APPROX_QUANTILES(trip_duration_sec / 60, 2)[OFFSET(1)] AS median_duration_min,
  MAX(trip_duration_sec / 60) AS max_duration_min
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

The average trip lasts around 12–15 minutes, with a median near 10 minutes. Most trips are short, which is likely because of urban commutes or quick errands, and while a few long rides (60+ minutes) stretch the max values.

### Q4. How do trip durations differ by rideable type?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  rideable_type,
  ROUND(AVG(trip_duration_sec) / 60, 2) AS avg_duration_min,
  ROUND(APPROX_QUANTILES(trip_duration_sec / 60, 2)[OFFSET(1)], 2) AS median_duration_min,
  COUNT(*) AS total_rides
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY rideable_type
ORDER BY avg_duration_min DESC;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=4, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

* Docked bikes have the longest average trip durations (31.3 min), likely due to older users or longer one-way trips.

* Classic bikes follow with moderate trip lengths (16.5 min), representing the main share of Bluebikes rides.

* Electric bikes show the shortest trips (14.4 min) and fewer total rides, suggesting they are newer in the system or used primarily for quick errands rather than long rides.

* Overall, trip duration decreases as bike type becomes more convenient and faster — consistent with electric bikes serving shorter, efficiency-focused trips.

### Q5. Does user type (member vs. casual) influence trip length and volume?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  user_type,
  ROUND(AVG(trip_duration_sec) / 60, 2) AS avg_duration_min,
  APPROX_QUANTILES(trip_duration_sec / 60, 2)[OFFSET(1)] AS median_duration_min,
  COUNT(*) AS total_rides
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY user_type
ORDER BY user_type;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=2, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

* Members take far more total rides but their trips are shorter on average (around 10–12 minutes).

* Casual riders make up a smaller share of rides but their trips are longer (15–20 minutes).
This pattern shows that members tend to use Bluebikes for daily commutes, while casual users ride for recreation or exploration, often during weekends or warmer months.

### Q6. How does ridership vary by day of week and user type?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH rides_by_day AS (
  SELECT
    FORMAT_DATE('%A', DATE(started_at)) AS day_of_week,
    user_type,
    COUNT(*) AS total_rides
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  GROUP BY day_of_week, user_type
)
SELECT
  day_of_week,
  user_type,
  total_rides
FROM rides_by_day
ORDER BY
  CASE day_of_week
    WHEN 'Monday' THEN 1
    WHEN 'Tuesday' THEN 2
    WHEN 'Wednesday' THEN 3
    WHEN 'Thursday' THEN 4
    WHEN 'Friday' THEN 5
    WHEN 'Saturday' THEN 6
    WHEN 'Sunday' THEN 7
  END;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=14, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

* Members: Ridership peaks Tuesday–Thursday, matching commuter usage.

* Casual riders: Usage spikes on Saturday and Sunday, consistent with recreational patterns.

* This demonstrates a clear behavioral split — weekdays for routine mobility, weekends for leisure.

### Q7. What times of day are busiest?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  EXTRACT(HOUR FROM started_at) AS hour_of_day,
  COUNT(*) AS total_rides
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
GROUP BY hour_of_day
ORDER BY hour_of_day;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=24, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

* Peak hours occur at 8–9 AM and 5–6 PM, reflecting commute periods.

* Midday (11 AM–3 PM) also shows moderate activity, driven mainly by casual riders.

* Early morning (before 6 AM) and late night (after 9 PM) rides are minimal.
This indicates clear rush-hour demand cycles for members, while casual users fill in midday recreational rides.

### Q8. Which stations handle the most trips?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH station_counts AS(
  SELECT start_station_name AS station, COUNT(*) AS trip_count
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  GROUP BY start_station_name

  UNION ALL

  SELECT end_station_name AS station, COUNT(*) trip_count
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  GROUP BY end_station_name)

SELECT station AS Top_10_busiest_station, SUM(trip_count) AS total_trips
FROM station_counts
GROUP BY station
ORDER BY total_trips DESC
LIMIT 10;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=10, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

* Demand is concentrated along the MIT–Harvard corridor, with 7 of the top 10 stations in Cambridge.

* MIT at Mass Ave / Amherst St is an outlier above 1.0M trips and should be prioritized for dock capacity, faster rebalancing, and higher e-bike availability.

* Central Square and Harvard Square show that transit-rich, mixed-use hubs sustain strong all-day demand.

### Q9. How common are round trips (returning to the same station)?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  user_type,
  COUNT(*) AS total_trips,
  COUNTIF(start_station_id = end_station_id) AS round_trips,
  ROUND(COUNTIF(start_station_id = end_station_id) / COUNT(*) * 100,2) AS pct_round_trips
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
WHERE start_station_id IS NOT NULL
  AND end_station_id IS NOT NULL
GROUP BY user_type
ORDER BY pct_round_trips DESC;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=2, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

* Casual riders have a much higher share of round trips (about 9%) compared to members (about 2.5%), suggesting that casual users are more likely to rent bikes for leisure or sightseeing rather than commuting.

* Members account for nearly three times as many total trips, reflecting strong habitual use tied to daily travel patterns and commutes.

* The clear behavioral split highlights that casual users value convenience and recreation, while members rely on the system as part of their regular mobility routine.

### Q10. What are the most common origin–destination pairs?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT start_station_name AS origin, end_station_name AS destination,COUNT(*) AS trip_count
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
WHERE start_station_name IS NOT NULL AND end_station_name IS NOT NULL
GROUP BY origin, destination
ORDER BY trip_count DESC;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=220980, table_html='<table border="1" class="dataframe table table-striped…

Computation deferred. Computation will process 15.2 MB

* The most frequent trips occur between stations around the MIT campus, especially along Mass Ave and Vassar St, showing dense intra-campus mobility and short-distance commuting.

* Bidirectional flows between MIT at Mass Ave / Amherst St and Beacon St at Massachusetts Ave indicate strong corridor use connecting Cambridge and Back Bay.

* The clustering of top routes within a small geographic area suggests a high local circulation network rather than dispersed citywide travel.

* These concentrated flows point to clear candidates for micro-rebalancing strategies and highlight stations that could benefit from additional docks or e-bikes.

### Q11. How are trip durations distributed?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH trips AS (
  SELECT
    user_type,
    ROUND(trip_duration_sec / 60.0, 1) AS duration_min
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  WHERE trip_duration_sec IS NOT NULL
    AND trip_duration_sec > 0
    AND trip_duration_sec < 3 * 60 * 60),

binned AS (
  SELECT
    user_type,
    CASE
      WHEN duration_min <= 5  THEN '0-5 min'
      WHEN duration_min <= 10 THEN '6-10 min'
      WHEN duration_min <= 15 THEN '11-15 min'
      WHEN duration_min <= 20 THEN '16-20 min'
      WHEN duration_min <= 30 THEN '21-30 min'
      WHEN duration_min <= 45 THEN '31-45 min'
      WHEN duration_min <= 60 THEN '46-60 min'
      ELSE '60+ min'
    END AS duration_range
  FROM trips),

bin_stats AS (
  SELECT
    duration_range,
    COUNT(*) AS trip_count,
    COUNTIF(user_type = 'member') AS member_trips,
    COUNTIF(user_type = 'casual') AS casual_trips
  FROM binned
  GROUP BY duration_range),

totals AS (
  SELECT SUM(trip_count) AS total_trips
  FROM bin_stats)

SELECT
  b.duration_range,
  b.trip_count,
  ROUND(b.trip_count * 100.0 / t.total_trips, 2) AS pct_of_trips,
  ROUND(b.member_trips * 100.0 / t.total_trips, 2) AS pct_member_in_all,
  ROUND(b.casual_trips * 100.0 / t.total_trips, 2) AS pct_casual_in_all
FROM bin_stats b
CROSS JOIN totals t
ORDER BY
  CASE b.duration_range
    WHEN '0-5 min'   THEN 1
    WHEN '6-10 min'  THEN 2
    WHEN '11-15 min' THEN 3
    WHEN '16-20 min' THEN 4
    WHEN '21-30 min' THEN 5
    WHEN '31-45 min' THEN 6
    WHEN '46-60 min' THEN 7
    ELSE 8
  END;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=8, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

* Short rides dominate the system, with six to ten minutes accounting for about 29% of all trips and eleven to fifteen minutes adding another 21%.

* Roughly three quarters of rides finish within twenty minutes, confirming that Bluebikes is used primarily for short urban hops.

* Members contribute most of the very short trips, while the casual share rises in the 21–30 minute and longer bins, which fits recreational and sightseeing behavior.

* Very long rides are rare, with trips longer than forty five minutes making up roughly 4–5% of total volume.

## 3.3 Weather and seasonal effects

### Q12. Does temperature affect ridership?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH temp_category AS (
  SELECT
    date,
    temp,
    CASE
      WHEN temp < 40 THEN 'Cold'
      WHEN temp BETWEEN 40 AND 80 THEN 'Mild'
      WHEN temp > 80 THEN 'Hot'
      ELSE 'Unknown'
    END AS temp_group
  FROM `ba775-fall25-b05.ba775_fall25_b05.weather_daily_summary`
  WHERE temp IS NOT NULL
),
daily_trips AS (
  SELECT
    DATE(started_at) AS trip_date,
    COUNT(*) AS trip_count
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  GROUP BY trip_date
)
SELECT
  t.temp_group,
  AVG(d.trip_count) AS avg_trips,
  COUNT(*) AS num_days
FROM daily_trips AS d
JOIN temp_category AS t
  ON d.trip_date = t.date
WHERE t.temp_group != 'Unknown'
GROUP BY t.temp_group
ORDER BY avg_trips DESC;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=3, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

This result includes 124 hot days, 1,950 mild days, and 593 cold days. As expected, when temperatures are higher than 80°F, the average daily ridership is the highest around 11,756 trips, followed by mild days with about 10,312 trips, and the lowest on cold days with around 4,160 trips. This indicates that biking activity increases as temperatures rise.

### Q13. What is the relationship between temperature ranges and the average number of Bluebike trips per day?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH trips_daily AS (
  SELECT DATE(started_at) AS trip_date, COUNT(*) AS trips
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  GROUP BY trip_date
),
joined AS (
  SELECT
    t.trip_date,
    w.temp AS temp,
    t.trips
  FROM trips_daily t
  LEFT JOIN `ba775-fall25-b05.ba775_fall25_b05.weather_daily_summary` w
    ON t.trip_date = w.date
  WHERE w.temp IS NOT NULL
)
SELECT
  CASE
    WHEN temp < 0 THEN '<0°C'
    WHEN temp BETWEEN 0 AND 4.9 THEN '0–4.9°C'
    WHEN temp BETWEEN 5 AND 9.9 THEN '5–9.9°C'
    WHEN temp BETWEEN 10 AND 14.9 THEN '10–14.9°C'
    WHEN temp BETWEEN 15 AND 19.9 THEN '15–19.9°C'
    WHEN temp BETWEEN 20 AND 24.9 THEN '20–24.9°C'
    ELSE '25°C+'
  END AS temp_bin,
  COUNT(*) AS days_in_bin,
  AVG(trips) AS avg_trips_per_day
FROM joined
GROUP BY temp_bin
ORDER BY MIN(temp);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=6, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes


By grouping daily demand into temperature ranges:

Usage is minimal below 5 °C, grows steadily through 10–20 °C, and peaks between 15–25 °C.

Beyond 25 °C, demand flattens or drops slightly.

Insight: The “comfort zone” for cycling is roughly 15–25 °C — neither too hot nor too cold.

### Q14. How does hazardous weather impact usage and trip duration?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT DISTINCT danger
FROM `ba775-fall25-b05.ba775_fall25_b05.weather_daily_summary`
ORDER BY danger;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=6, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH trips_by_day AS (
  SELECT
    DATE(started_at) AS trip_date,
    COUNT(*) AS trips,
    AVG(IF(trip_duration_sec BETWEEN 60 AND 8*3600, trip_duration_sec, NULL)) AS avg_trip_duration_sec
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  GROUP BY trip_date
),
joined AS (
  SELECT
    w.danger,
    t.trips,
    t.avg_trip_duration_sec
  FROM trips_by_day t
  JOIN `ba775-fall25-b05.ba775_fall25_b05.weather_daily_summary` w
    ON t.trip_date = w.date
  WHERE w.danger IS NOT NULL
)
SELECT
  danger,
  ROUND(AVG(trips), 2) AS avg_daily_trips,
  ROUND(AVG(avg_trip_duration_sec), 2) AS avg_trip_duration_sec,
  ROUND(AVG(avg_trip_duration_sec) / 60, 2) AS avg_trip_duration_min
FROM joined
GROUP BY danger
ORDER BY CASE WHEN danger = 'none' THEN 0 ELSE 1 END, danger;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=6, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes


*   Normal (“none”) weather shows the highest overall
activity baseline, with about 9,669 daily trips and an average duration of about 17 minutes.
*   Heavy rain and wind also lower ridership to about 6,800–7,100 trips, with slightly shorter durations 15–16 minutes.

*   Cold and snow conditions reduce both trip counts and duration. Daily trips fall to around 1500 to 1600, and average trip time drops to about 13 minutes, indicating that riders avoid long exposure to freezing weather.
*   However, storm days show slightly higher average trips of 10,178 and the longest average duration 17.5 minutes, which could reflect limited data.

### Q15. How does ridership vary by season?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
WITH trips_with_season AS (
  SELECT
    DATE(started_at) AS trip_date,
    EXTRACT(MONTH FROM started_at) AS month,
    CASE
      WHEN EXTRACT(MONTH FROM started_at) IN (12, 1, 2) THEN 'Winter'
      WHEN EXTRACT(MONTH FROM started_at) IN (3, 4, 5) THEN 'Spring'
      WHEN EXTRACT(MONTH FROM started_at) IN (6, 7, 8) THEN 'Summer'
      WHEN EXTRACT(MONTH FROM started_at) IN (9, 10, 11) THEN 'Fall'
    END AS season
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
)
SELECT
  season,
  COUNT(*) AS total_trips,
  ROUND(COUNT(*) / SUM(COUNT(*)) OVER() * 100, 2) AS pct_of_total
FROM trips_with_season
GROUP BY season
ORDER BY total_trips DESC;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=4, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

This ridership shows a clear seasonal pattern, peaking in summer with about 8.98 million trips (36%) of annual activity. Usage declines slightly in fall (31%) and further in spring (22%), while winter accounts for the lowest ridership at 2.65 million trips (11%). This trend highlights how warmer seasons strongly encourage biking activity, whereas colder months significantly reduce usage.

## 3.4 E‑bike adoption and station dynamics

### Q16. What proportion of trips use e‑bikes after their December 2023 introduction?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
-- Monthly proportion of e-bike trips since December 2023
SELECT
  FORMAT_DATE('%Y', DATE(started_at)) AS year,
  COUNTIF(rideable_type = 'electric_bike') AS e_bike_trips,
  COUNT(*) AS total_trips,
  ROUND(100 * COUNTIF(rideable_type = 'electric_bike') / COUNT(*), 2) AS e_bike_percent
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
WHERE DATE(started_at) >= '2023-12-01'
GROUP BY year
ORDER BY year;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=3, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In 2023, e-bikes accounted for only 1.72% of total trips, which is expected since they were newly introduced at the end of the year. In 2024, e-bike usage increased sharply to 25.46%, reflecting strong adoption after their launch. For 2025, the share reached 27.82% as of September, showing a slight continued increase, though the data is limited to part of the year.

### Q17. Do members or casual riders use e‑bikes more?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT
  user_type,
  rideable_type,
  COUNT(*) AS total_trips,
  ROUND(100 * COUNT(*) / SUM(COUNT(*)) OVER (PARTITION BY user_type), 2) AS percent_within_user_type
FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
WHERE rideable_type IS NOT NULL AND rideable_type NOT IN ('N/A', '')
GROUP BY user_type, rideable_type
ORDER BY user_type, percent_within_user_type DESC;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=6, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

For causal riders, 56.57% of trips use classic bikes, followed by 25.98% docked bikes and only 17.45% use electric bike.

For members which shows a similar pattern. 53.8% of trips are on classic bikes, 26.35% on docked bikes, and only 19.85 on electric bikes.

Overall, classic bikes dominate usage across both groups, while electric bikes remain the least used type.

### Q18. Where are e‑bike trips starting compared with classic bikes?

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
SELECT *
FROM (
  SELECT
    rideable_type,
    start_station_name,
    COUNT(*) AS total_trips,
    ROW_NUMBER() OVER (PARTITION BY rideable_type ORDER BY COUNT(*) DESC) AS rank
  FROM `ba775-fall25-b05.ba775_fall25_b05.trips_clean`
  WHERE rideable_type IN ('electric_bike', 'classic_bike')
  GROUP BY rideable_type, start_station_name
)
WHERE rank <= 10
ORDER BY rideable_type, rank;

""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=20, table_html='<table border="1" class="dataframe table table-striped tab…

Computation deferred. Computation will process 0 Bytes

The top starting stations for e-bikes and classic bikes show a high degree of overlap, suggesting that riders of both types follow similar travel patterns. Stations such as MIT at Mass Ave / Amherst St, Central Square at Mass Ave / Essex St, and Harvard Square at Mass Ave / Dunster rank among the top three for both bike types, indicating that these locations are major hubs for all riders.

# **4. Conclusions**

## 4.1 Insights

1. **Demand concentrates along the MIT–Harvard corridor and a few hub stations.**
   Action: Increase dock counts and e-bike inventory at these hubs, set higher priority rebalancing loops, and designate nearby overflow stations to absorb surges.

2. **Most trips are under 20 minutes, with a large peak at 6–10 minutes.**
   Action: Tune pricing and messaging around short, quick trips; ensure fast turn-around at high-throughput docks; schedule maintenance and rebalancing outside these peak windows.

3. **Casual riders make many more round trips than members.**
   Action: Add visitor-friendly features at tourist and campus hubs (clear return guidance, loop suggestions, wayfinding), and run targeted conversion offers that nudge frequent casuals to memberships.

4. **Top origin–destination flows cluster within a small area around MIT, Central, and Beacon.**
   Action: Create micro-rebalancing circuits between these specific station pairs, pre-position e-bikes for the dominant directions, and pilot dynamic caps on dock availability to keep return capacity open.

5. **Members deliver the majority of volume and skew to one-way, commute-style rides.**
   Action: Protect commute reliability with morning and evening service levels, add capacity near transit nodes, and monitor “trips per dock” to identify stations where a permanent dock expansion is justified.

6. **Average trip times shorten under windy, rainy, or snowy conditions.** Action: riders prefer favor quick, necessity-based trips instead of leisure rides, so optimize rebalancing cycles for short-duration turnover; delay maintenance dispatches until calmer weather conditions.

7. **Hazardous weather correlates with higher late-return and maintenance events.**
Action: Introduce predictive maintenance alerts based on weather forecasts and reinforce communication about safe-riding protocols.

## 4.2 Risks

* The analysis is intentionally straightforward and outcome focused, which increases clarity and impact but may not appear as complex or “impressive” to some stakeholders.

* Several columns contain many nulls, and while they were included in EDA for exploration, they should be imputed or removed in future analysis; if any are critical to conclusions, we will add explicit disclaimers.

* Handling geospatial operations purely in SQL is more difficult than in Python, so we may encounter technical hurdles that require switching tools or adding preprocessing steps.

## 4.3 Next Steps

* Integrate neighborhood boundaries and aggregate trips by start and end locations to compare ridership, utilization, and e-bike share across the city.

* Enrich stations with nearby landmarks and activity centers (MBTA stops, campuses, parks, major employers) to assess how proximity to key destinations shapes demand.

* Develop interactive dashboards powered by Tableau views to track ridership, station performance, user mix, weather impacts, and e-bike adoption for operational monitoring.

* Create interactive maps that visualize station usage, neighborhood-level demand, and origin–destination flows to reveal hot spots, gaps, and rebalancing opportunities.

# **5. References**


   

*   Bluebikes in 2020 https://storymaps.arcgis.com/stories/0f5fc6ed107d4c0491d24051eed77ff9

*   Exploring BlueBikes Boston: An Analysis Using Tableau https://medium.com/@aliaformo/exploring-bluebikes-boston-an-analysis-using-tableau-4a6efc3dd407


*   Bike Share Expansion 2024-2025 https://www.boston.gov/departments/transportation/bike-share-expansion-2024-2025




# **6. Generative AI Disclosure**

*   We used ChatGPT to assist with uploading the trips data, as it spans over 89 CSV files with occasional column-name/schema differences, so we used Cloud Shell (gsutil + bq).

*   We used ChatGPT to align querying style and syntax across notebooks written by different team members. It helped standardize variable names.

*   We used ChatGPT to improve grammar and tighten the wording of documentation, analysis text, and conclusions. Every numeric claim was cross-checked against our tables and figures, and any AI-edited text was retained only if it accurately reflected our findings.

*  ChatGPT was occasionally used to interpret BigQuery error messages, but all query logic and analytical decisions were made by the team.